In [1]:
import os

# Define the directory where you want to save the dataset
dataset_dir = '/content/drive/MyDrive/Task5'

# Ensure the directory exists, create it if necessary
os.makedirs(dataset_dir, exist_ok=True)

# Change to the dataset directory
os.chdir(dataset_dir)

# Download the data
print("Downloading the data...")
!wget -P {dataset_dir} http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
print("Dataset downloaded!")

# Extract the data
print("Extracting data..")
!tar xzvf food-101.tar.gz > /dev/null 2>&1
print("Extraction done!")

--2024-05-22 14:31:18--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2024-05-22 14:31:18--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘/content/drive/MyDrive/Task5/food-101.tar.gz’

food-101.tar.gz     100%[===================>]   4.65G  21.8MB/s    in 3m 41s  

2024-05-22 14:35:00 (21.6 MB/s) - ‘/content/drive/MyDrive/Task5/food-101.tar.gz’ saved [4996278331/4996278331]

Dataset downloaded!
Extracting data..
Extraction done!


In [2]:
pip install tensorflow numpy pandas matplotlib tqdm

In [3]:
import zipfile
from tqdm import tqdm
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Define paths
extract_path = '/content/drive/MyDrive/Task5/food-101'

# Assuming the images are extracted to 'images' folder within 'Food-101'
image_dir = os.path.join(extract_path, 'images')

# Function to load images and labels
def load_images_from_directory(directory, img_size=(64, 64)):
    images = []
    labels = []
    classes = os.listdir(directory)
    for class_name in tqdm(classes, desc="Loading images"):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            for img_file in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_file)
                try:
                    img = cv2.imread(img_path)
                    if img is not None:
                        img = cv2.resize(img, img_size)
                        images.append(img)
                        labels.append(class_name)
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")
    return np.array(images), np.array(labels)

# Load the data
X, y = load_images_from_directory(image_dir)

# Check if images were loaded
print(f"Number of images loaded: {len(X)}")

# Encode labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
num_classes = len(np.unique(y_encoded))
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)


Loading images: 100%|██████████| 101/101 [06:02<00:00,  3.59s/it]


Number of images loaded: 101000


In [45]:
# Define the CNN model
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create the model
input_shape = X_train.shape[1:]  # (64, 64, 3)
model = create_cnn_model(input_shape, num_classes)

# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2, batch_size=32)
# Save the trained model
model.save('food_recognition_model.h5')
print("Model saved as 'food_recognition_model.h5'")

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')

Epoch 1/100
2020/2020 [==============================] - 18s 8ms/step - loss: 4.7329 - accuracy: 0.0095 - val_loss: 4.6160 - val_accuracy: 0.0081
Epoch 2/100
2020/2020 [==============================] - 13s 6ms/step - loss: 4.6157 - accuracy: 0.0101 - val_loss: 4.6164 - val_accuracy: 0.0081
Epoch 3/100
2020/2020 [==============================] - 13s 6ms/step - loss: 4.6207 - accuracy: 0.0099 - val_loss: 4.6188 - val_accuracy: 0.0114
Epoch 4/100
2020/2020 [==============================] - 14s 7ms/step - loss: 4.5692 - accuracy: 0.0169 - val_loss: 4.4812 - val_accuracy: 0.0238
Epoch 5/100
2020/2020 [==============================] - 14s 7ms/step - loss: 4.4947 - accuracy: 0.0249 - val_loss: 4.4185 - val_accuracy: 0.0336
Epoch 6/100
2020/2020 [==============================] - 13s 6ms/step - loss: 4.4195 - accuracy: 0.0347 - val_loss: 4.3312 - val_accuracy: 0.0507
Epoch 7/100
2020/2020 [==============================] - 13s 6ms/step - loss: 4.3480 - accuracy: 0.0465 - val_loss: 4.2495 -

In [5]:
# Assuming we have a dictionary of calories per food item
calorie_data ={'apple_pie': 0,
 'baby_back_ribs': 1,
 'baklava': 2,
 'beef_carpaccio': 3,
 'beef_tartare': 4,
 'beet_salad': 5,
 'beignets': 6,
 'bibimbap': 7,
 'bread_pudding': 8,
 'breakfast_burrito': 9,
 'bruschetta': 10,
 'caesar_salad': 11,
 'cannoli': 12,
 'caprese_salad': 13,
 'carrot_cake': 14,
 'ceviche': 15,
 'cheese_plate': 16,
 'cheesecake': 17,
 'chicken_curry': 18,
 'chicken_quesadilla': 19,
 'chicken_wings': 20,
 'chocolate_cake': 21,
 'chocolate_mousse': 22,
 'churros': 23,
 'clam_chowder': 24,
 'club_sandwich': 25,
 'crab_cakes': 26,
 'creme_brulee': 27,
 'croque_madame': 28,
 'cup_cakes': 29,
 'deviled_eggs': 30,
 'donuts': 31,
 'dumplings': 32,
 'edamame': 33,
 'eggs_benedict': 34,
 'escargots': 35,
 'falafel': 36,
 'filet_mignon': 37,
 'fish_and_chips': 38,
 'foie_gras': 39,
 'french_fries': 40,
 'french_onion_soup': 41,
 'french_toast': 42,
 'fried_calamari': 43,
 'fried_rice': 44,
 'frozen_yogurt': 45,
 'garlic_bread': 46,
 'gnocchi': 47,
 'greek_salad': 48,
 'grilled_cheese_sandwich': 49,
 'grilled_salmon': 50,
 'guacamole': 51,
 'gyoza': 52,
 'hamburger': 53,
 'hot_and_sour_soup': 54,
 'hot_dog': 55,
 'huevos_rancheros': 56,
 'hummus': 57,
 'ice_cream': 58,
 'lasagna': 59,
 'lobster_bisque': 60,
 'lobster_roll_sandwich': 61,
 'macaroni_and_cheese': 62,
 'macarons': 63,
 'miso_soup': 64,
 'mussels': 65,
 'nachos': 66,
 'omelette': 67,
 'onion_rings': 68,
 'oysters': 69,
 'pad_thai': 70,
 'paella': 71,
 'pancakes': 72,
 'panna_cotta': 73,
 'peking_duck': 74,
 'pho': 75,
 'pizza': 76,
 'pork_chop': 77,
 'poutine': 78,
 'prime_rib': 79,
 'pulled_pork_sandwich': 80,
 'ramen': 81,
 'ravioli': 82,
 'red_velvet_cake': 83,
 'risotto': 84,
 'samosa': 85,
 'sashimi': 86,
 'scallops': 87,
 'seaweed_salad': 88,
 'shrimp_and_grits': 89,
 'spaghetti_bolognese': 90,
 'spaghetti_carbonara': 91,
 'spring_rolls': 92,
 'steak': 93,
 'strawberry_shortcake': 94,
 'sushi': 95,
 'tacos': 96,
 'takoyaki': 97,
 'tiramisu': 98,
 'tuna_tartare': 99,
 'waffles': 100}

# Map labels to calories
y_calories = np.array([calorie_data[label] for label in y])

# Split the data for calorie prediction
X_train_cal, X_test_cal, y_train_cal, y_test_cal = train_test_split(X, y_calories, test_size=0.2, random_state=42)

# Define a simple regression model for calorie estimation
def create_regression_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

# Create the calorie estimation model
calorie_model = create_regression_model(input_shape)

# Train the model
calorie_model.fit(X_train_cal, y_train_cal, epochs=15, validation_split=0.2, batch_size=32)
# Save the trained model
calorie_model.save('calorie_model.h5')
print("Model saved as 'calorie_model.h5'")

# Evaluate the model
mse, mae = calorie_model.evaluate(X_test_cal, y_test_cal)
print(f'Test MAE for calorie estimation: {mae}')


Epoch 1/15
2020/2020 [==============================] - 14s 6ms/step - loss: 2133.7620 - mae: 29.2468 - val_loss: 877.9740 - val_mae: 25.1475
Epoch 2/15
2020/2020 [==============================] - 12s 6ms/step - loss: 1102.0385 - mae: 27.5393 - val_loss: 915.0786 - val_mae: 25.4248
Epoch 3/15
2020/2020 [==============================] - 12s 6ms/step - loss: 1087.3279 - mae: 27.2926 - val_loss: 890.0829 - val_mae: 25.2298
Epoch 4/15
2020/2020 [==============================] - 12s 6ms/step - loss: 1053.4735 - mae: 26.9200 - val_loss: 867.3701 - val_mae: 25.0442
Epoch 5/15
2020/2020 [==============================] - 12s 6ms/step - loss: 1035.4448 - mae: 26.6871 - val_loss: 884.6553 - val_mae: 25.1457
Epoch 6/15
2020/2020 [==============================] - 12s 6ms/step - loss: 1005.4316 - mae: 26.3221 - val_loss: 868.7488 - val_mae: 25.0572
Epoch 7/15
2020/2020 [==============================] - 12s 6ms/step - loss: 983.1412 - mae: 26.0237 - val_loss: 911.3525 - val_mae: 25.4626
Epoch 8